# **Árboles de Decisión y Random Forest**

En este notebook se entrena un modelo de **árbol de regresión (de Decisión y Random Forest)** para predecir la esperanza de vida a partir del dataset procesado (en el notebook `EDA_processed.ipynb`).

En este notebook vamos a trabajar con **modelos basados en árboles**:

- **Árbol de Decisión (Decision Tree Regressor)**: sencillo de interpretar, pero propenso al sobreajuste.  
- **Random Forest Regressor**: un ensamblado de árboles mediante bagging que reduce la varianza y mejora la generalización.  

Además, exploraremos el concepto de **overfitting** con estos modelos.

Del EDA_processed.ipynb exportamos dos datasets:

- features_no_scaling.csv → para árboles, Random Forest, XGBoost.

- features_scaled.csv → para modelos sensibles a la escala (Regresión Lineal, KNN, SVM...).
Además del target_y.csv.

Aquí usaremos el no escalado `features_no_scaling.csv`. 


## **Paso 1. Importar librerías y cargar datasets procesados**

En este bloque cargamos las librerías necesarias:

- **pandas** → manejo de datos.  
- **scikit-learn** → para la división del dataset (`train_test_split`), construcción de modelos de regresión (`DecisionTreeRegressor`, `RandomForestRegressor`) y cálculo de métricas (`MAE`, `RMSE`, `R²`).  
- **plot_tree** → permite visualizar gráficamente un árbol de decisión entrenado.  
- **matplotlib / seaborn / numpy** → apoyo para gráficos y cálculos numéricos.

Con estas herramientas podremos entrenar, evaluar y visualizar el comportamiento de los modelos basados en árboles.

En este paso también importamos los datasets que ya preparamos en el notebook **EDA_processed**:

- `features_no_scaling.csv` → variables predictoras sin escalar (adecuadas para modelos basados en árboles).
- `target_y.csv` → variable objetivo (esperanza de vida).

Aquí **no usamos el dataset escalado**, ya que árboles y Random Forest no necesitan normalización de variables.  
Al final imprimimos las dimensiones de `X` e `y` para confirmar que todo está alineado.



In [ ]:
# ===================================
# 1. Importar librerías
# ===================================

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


# ===================================
# 2. Cargar datasets procesados no escalados 
# ===================================
X_ns = pd.read_csv("../data/processed/features_no_scaling.csv")
y = pd.read_csv("../data/processed/target_y.csv").squeeze()  # lo convertimos a Serie




## **Paso 2. División en entrenamiento y validación**

En este bloque:

- Separamos el dataset en **conjunto de entrenamiento (80%)** y **conjunto de validación (20%)**.  
Esto nos permite entrenar el modelo con una parte de los datos y luego evaluar su rendimiento en datos que no ha visto antes.


De esta manera, podremos comparar los algoritmos de forma justa y evitar errores de entrenamiento.

- x = todas las columnas salvo la variable objetivo.
- y = la variable objetivo.
- train_test_split divide los datos en:
  - x_train, y_train (80%) → para entrenar el modelo.
  - x_valid, y_valid (20%) → para validar el modelo. 
  - random_state=42 → asegura que la división sea reproducible.

Este paso es obligatorio para aseguramos consistencia y poder comparar con en el resto de modelos.


In [ ]:
# ===================================
# 2. División train / validación
# ===================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X_ns, y, test_size=0.2, random_state=42
)

print("Tamaño entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño validación:", X_valid.shape, y_valid.shape)


## **Paso 3. Árbol de Decisión baseline**

En este paso entrenamos un **Árbol de Decisión** con los parámetros por defecto y evaluamos las métricas.

Un árbol de decisión divide los datos en nodos según la variable que mejor reduce la **impureza** (en regresión, normalmente el **MSE** dentro de cada nodo).  
Este baseline nos sirve como punto de partida para comparar con modelos más complejos como **Random Forest** o **XGBoost**.

Métricas que calculamos:
- **RMSE (Root Mean Squared Error):** penaliza más los errores grandes.  
- **MAE (Mean Absolute Error):** error promedio en unidades originales (años de esperanza de vida).  
- **R²:** proporción de varianza explicada (0 a 1 → cuanto más alto, mejor).



In [ ]:
# ===================================
# 3. Árbol de Decisión (baseline)
# ===================================

tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

preds_tree = tree.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, preds_tree))
mae = mean_absolute_error(y_valid, preds_tree)
r2 = r2_score(y_valid, preds_tree)

print("📊 Decision Tree (baseline)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


## **Paso 4. Overfitting en Árboles de Decisión: curva de aprendizaje con max_depth**


Los **árboles de decisión** tienden a crecer hasta memorizar los datos de entrenamiento, lo que provoca **overfitting**.  

Para analizarlo, variamos la **profundidad máxima (`max_depth`)** del árbol y graficamos los errores (RMSE) en:  
- **Train (línea azul):** error sobre los datos de entrenamiento.  
- **Valid (línea naranja):** error sobre datos no vistos (validación).  


### 📊 Interpretación de la curva de validación (Decision Tree)


En este gráfico vemos dos curvas:  
- **Train RMSE** → error en los datos de entrenamiento.  
- **Valid RMSE** → error en los datos de validación. 


🔎 **Qué significa la separación de las curvas:**
- Si **Train RMSE ≪ Valid RMSE**, el modelo está sobreajustando (overfitting).  
- Si **ambos errores son altos**, el modelo está infraajustando (underfitting).  
- El punto ideal es cuando **las curvas están lo más cercanas posible una de otra** y en un nivel bajo de error.  

Esto indica un buen equilibrio entre **sesgo (bias)** y **varianza (variance)**.

 
⚙️ **Efecto de los hiperparámetros**:
- **`max_depth ↑`** o **aumenta** → el árbol crece más → memoriza mejor el train (↓ error en train) pero aumenta el overfitting. El error en train baja mucho, pero en validación empieza a subir a partir de cierto punto (signo de **sobreajuste**). 
- **`max_depth ↓`** o es **muy bajo** → el árbol o el modelo es demasiado simple (**underfitting**) → puede aumentar el error en train (errores altos), pero mejora la generalización.  


Lo que buscamos es un **compromiso entre complejidad y generalización**. 


In [ ]:
# ===================================
# 4. Overfitting en Árboles de Decisión
# ===================================

train_errors, valid_errors = [], []
depths = range(1, 21)

for d in depths:
    model = DecisionTreeRegressor(max_depth=d, random_state=42)
    model.fit(X_train, y_train)
    preds_train = model.predict(X_train)
    preds_valid = model.predict(X_valid)
    
    train_errors.append(np.sqrt(mean_squared_error(y_train, preds_train)))
    valid_errors.append(np.sqrt(mean_squared_error(y_valid, preds_valid)))

plt.figure(figsize=(8,5))
plt.plot(depths, train_errors, label="Train RMSE", marker="o")
plt.plot(depths, valid_errors, label="Valid RMSE", marker="o")
plt.xlabel("Profundidad del árbol (max_depth)")
plt.ylabel("RMSE")
plt.title("Overfitting en Decision Tree")
plt.legend()
plt.show()


## **Paso 5. Random Forest (baseline)**

Ahora entrenamos un Random Forest y evaluamos sus métricas.  
Este modelo combina muchos árboles entrenados sobre subconjuntos aleatorios de los datos, lo que reduce la varianza.


Un **Random Forest** es un ensamble de árboles de decisión, entrena **muchos árboles de decisión** en paralelo, cada uno con:
- Un subconjunto aleatorio de observaciones.  
- Un subconjunto aleatorio de variables.  

Esto introduce **diversidad en los árboles** y permite que, al promediar sus resultados, el modelo:
- Reduzca la varianza (menos overfitting que un solo árbol).  
- Generalice mejor en validación.

- Cada árbol se entrena sobre un subconjunto aleatorio de datos y de variables (**bagging**).  
- Al combinar muchos árboles, se reducen los problemas de **overfitting** que vimos en el árbol simple.  
- Es más robusto y suele mejorar las métricas de validación.



⚙️ El hiperparámetro clave es `n_estimators`:
- **Si `n_estimators ↑`** → más árboles → resultados más estables pero mayor tiempo de cómputo.  
- **Si `n_estimators ↓`** → menos árboles → más rápido, pero menos robusto.  



En este baseline usamos `n_estimators=200` árboles con parámetros por defecto.  


In [ ]:

# ===================================
# 5. Random Forest (baseline)
# ===================================

rf = RandomForestRegressor(random_state=42, n_estimators=200)
rf.fit(X_train, y_train)

preds_rf = rf.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, preds_rf))
mae = mean_absolute_error(y_valid, preds_rf)
r2 = r2_score(y_valid, preds_rf)

print("📊 Random Forest (baseline)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


## **Paso 6. Importancia de Variables**

Los modelos basados en árboles permiten calcular la **importancia de las variables**:  
permiten medir qué tan importante es cada variable, cuánto contribuye cada predictor a reducir el error en los nodos de decisión. 

Una ventaja de los modelos basados en árboles es que permiten calcular la **importancia de las variables**.  

En el caso de **Random Forest**, la importancia se calcula promediando las reducciones de impureza de todos los árboles del ensamble.  

- Una **importancia alta** indica que esa variable es muy influyente en las predicciones.  
- Una **importancia baja** significa que la variable apenas aporta información.  


💡 Recordatorio:  
- El hiperparámetro `n_estimators` indica **cuántos árboles de decisión** se entrenan en el Random Forest.  
- A más árboles, el modelo suele ser más robusto y estable, aunque también más costoso en tiempo de entrenamiento.  
- Un valor típico es 100–500 árboles, y aquí usamos `n_estimators=200`.  

En este gráfico mostramos el **Top 20**.  
Esto nos ayuda a:
- Entender mejor el dataset.  
- Posibles reducciones de dimensionalidad (quedarnos con las más influyentes).  
- Interpretar el modelo y explicar los resultados.  

In [ ]:
# ===================================
# 6. Importancia de Variables
# ===================================

importances = pd.DataFrame({
    "Variable": X_train.columns,
    "Importancia": rf.feature_importances_
}).sort_values(by="Importancia", ascending=False)

plt.figure(figsize=(8,10))
sns.barplot(
    x="Importancia", 
    y="Variable", 
    data=importances.head(20), 
    hue="Variable",          
    dodge=False, 
    legend=False,            # ocultamos la leyenda
    palette="Blues_r"
)
plt.title("Top 20 variables más importantes (Random Forest)")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.show()

importances.head(20)


## **Paso 7. Ajuste de hiperparámetros en Random Forest**


Hasta ahora hemos usado un **Random Forest baseline** con parámetros por defecto (`n_estimators=200`).  
Aunque este modelo ya mejora claramente al árbol de decisión simple, podemos explorar cómo afecta variar algunos parámetros clave:

- **`n_estimators`**: número de árboles en el ensamble. Más árboles reducen la varianza, pero aumentan el coste computacional.  
- **`max_depth`**: profundidad máxima de los árboles. Controla la complejidad del modelo y previene overfitting.  
- **`min_samples_split`**: número mínimo de muestras necesarias para dividir un nodo. Valores más altos hacen que los árboles sean más simples.  

En este paso vamos a hacer una **búsqueda manual** variando `n_estimators` y observando cómo cambia el RMSE en train y validación.  

📊 **Qué esperamos ver en el gráfico**:  
- A medida que aumenta `n_estimators`, el error de validación debería estabilizarse.  
- Si las curvas de train y validación están cercanas y planas, significa que el modelo generaliza bien.  
- Si hay una gran diferencia (train mucho mejor que validación), significa que aún hay cierto overfitting.  




In [ ]:
# ===================================
# 7. Ajuste de hiperparámetros en Random Forest
# ===================================
train_errors, valid_errors = [], []
n_estimators_range = range(10, 310, 20)  # probamos de 10 a 300 árboles
## n_estimators_range = [50, 100, 200, 300, 500]

for n in n_estimators_range:
    rf_model = RandomForestRegressor(random_state=42, n_estimators=n)
    rf_model.fit(X_train, y_train)
    
    preds_train = rf_model.predict(X_train)
    preds_valid = rf_model.predict(X_valid)
    
    train_errors.append(np.sqrt(mean_squared_error(y_train, preds_train)))
    valid_errors.append(np.sqrt(mean_squared_error(y_valid, preds_valid)))

# Gráfico de curva de aprendizaje en función de n_estimators
plt.figure(figsize=(8,5))
plt.plot(n_estimators_range, train_errors, label="Train RMSE", marker="o")
plt.plot(n_estimators_range, valid_errors, label="Valid RMSE", marker="o")
plt.xlabel("Número de árboles (n_estimators)")
plt.ylabel("RMSE")
plt.title("Curva de validación - Random Forest")
plt.legend()
plt.show()


## **Paso 8. GridSearchCV para optimización de Random Forest**

Hasta ahora usamos hiperparámetros por defecto.  
Con **GridSearchCV** buscamos la mejor combinación de parámetros mediante validación cruzada.  

⚙️ Principales hiperparámetros:
- **`max_depth`**  
  - ↑ profundidad → más complejo, riesgo de overfitting.  
  - ↓ profundidad → más simple, riesgo de underfitting.  

- **`min_samples_split`**  
  - ↑ valor → obliga a que los nodos tengan más datos para dividirse → árbol más simple.  
  - ↓ valor → más divisiones → riesgo de overfitting.  

- **`min_samples_leaf`**  
  - ↑ valor → hojas más grandes → árbol más suave y general.  
  - ↓ valor → hojas con muy pocos datos → riesgo de memorizar.  

- **`n_estimators`**  
  - ↑ más árboles → modelo más robusto, pero más lento.  
  - ↓ menos árboles → más rápido, pero más inestable.  

👉 El objetivo es **acercar las curvas de train y validación** y reducir el RMSE. 

📖 Explicación de loq ue hacemos con GridSearchCV:

  - GridSearchCV prueba todas las combinaciones de hiperparámetros en param_grid.
  - Usa validación cruzada (cv=5) para evaluar el rendimiento medio y reducir el riesgo de overfitting.
  - El mejor modelo (best_estimator_) se reentrena con los mejores parámetros encontrados.
  - Luego lo evaluamos en nuestro conjunto de validación independiente (hold-out) para confirmar.

💡 Esto nos permite justificar mejor frente a XGBoost + Optuna:

  - GridSearchCV = búsqueda exhaustiva (más lento).
  - Optuna = búsqueda inteligente con optimización bayesiana (más rápido y escalable).


In [ ]:
# ===================================
# 8. Optimización de Random Forest con GridSearchCV
# ===================================
from sklearn.model_selection import GridSearchCV

# Definimos el modelo base
rf = RandomForestRegressor(random_state=42)

# Definimos la rejilla de hiperparámetros a explorar
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

# Configuramos GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",  # usamos RMSE
    cv=5,                                  # validación cruzada 5-fold
    n_jobs=-1,                             # usar todos los núcleos
    verbose=2
)

# Entrenamos
grid_search.fit(X_train, y_train)

print("✅ Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

print("\n📊 Mejor score (RMSE validación):")
print(-grid_search.best_score_)

# Evaluación final en nuestro conjunto de validación hold-out
best_rf = grid_search.best_estimator_
preds_best_rf = best_rf.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, preds_best_rf))
mae = mean_absolute_error(y_valid, preds_best_rf)
r2 = r2_score(y_valid, preds_best_rf)

print("\n📊 Random Forest Optimizado (validación hold-out)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


## **Paso 9. Heatmap de resultados de GridSearchCV**

Visualización de resultados de GridSearchCV con un heatmap.


💡 Este heatmap muestra cómo cambian los errores medios de validación cruzada (RMSE) en función de dos hiperparámetros clave:

- `n_estimators` = número de árboles.
- `max_depth` = profundidad máxima de cada árbol.
- colores más claros → menor error → mejor combinación.

In [ ]:
# ===================================
# 9. Visualización de resultados de GridSearchCV
# ===================================
results = pd.DataFrame(grid_search.cv_results_)

# Extraemos solo los parámetros y la media de la métrica (neg RMSE)
pivot_table = results.pivot_table(
    values="mean_test_score",
    index="param_max_depth",
    columns="param_n_estimators"
)

plt.figure(figsize=(8,6))
sns.heatmap(
    -pivot_table,  # convertimos de "neg RMSE" a RMSE positivo
    annot=True, fmt=".3f", cmap="Blues"
)
plt.title("Heatmap RMSE promedio (GridSearchCV - Random Forest)")
plt.xlabel("n_estimators")
plt.ylabel("max_depth")
plt.show()


## **Paso 10. Conclusiones Árboles de Decisión vs. Random Forest Regressor**

- **Árbol de Decisión (baseline):**
  - Modelo muy interpretable y rápido de entrenar.
  - Sin embargo, tiende a **sobreajustar** cuando la profundidad aumenta.
  - Vimos en la curva de validación cómo el error en train bajaba mucho, pero el de validación empezaba a subir → overfitting.

- **Random Forest (baseline):**
  - Combina muchos árboles entrenados en subconjuntos de datos (técnica de **bagging**).
  - Esto reduce la varianza y mejora la generalización.
  - Métricas mejores que el árbol simple en validación.

- **Random Forest optimizado con GridSearchCV:**
  - Ajustamos automáticamente varios hiperparámetros:
    - `n_estimators`: número de árboles.
    - `max_depth`: controla la complejidad.
    - `min_samples_split` y `min_samples_leaf`: evitan nodos demasiado pequeños.
  - GridSearchCV hace una **búsqueda exhaustiva** de combinaciones y selecciona la mejor mediante validación cruzada.
  - El heatmap nos permitió ver cómo ciertas combinaciones (`n_estimators` altos + `max_depth` medio) ofrecen los mejores resultados.
  - Resultado: mejora del RMSE y R² respecto al baseline.

👉 Conclusión final: **Random Forest optimizado** ofrece un mejor equilibrio entre precisión y robustez frente a sobreajuste, siendo superior al árbol de decisión simple.


## **Comparativa de modelos:** 
### Árbol de Decisión, Random Forest baseline y Random Forest optimizado


In [ ]:
# ===================================
# Comparativa final de métricas
# ===================================

# Guardamos las métricas en un diccionario
results = {
    "Decision Tree": {
        "RMSE": np.sqrt(mean_squared_error(y_valid, preds_tree)),
        "MAE": mean_absolute_error(y_valid, preds_tree),
        "R²": r2_score(y_valid, preds_tree)
    },
    "Random Forest (baseline)": {
        "RMSE": np.sqrt(mean_squared_error(y_valid, preds_rf)),
        "MAE": mean_absolute_error(y_valid, preds_rf),
        "R²": r2_score(y_valid, preds_rf)
    },
    "Random Forest (optimizado)": {
        "RMSE": np.sqrt(mean_squared_error(y_valid, grid_search.best_estimator_.predict(X_valid))),
        "MAE": mean_absolute_error(y_valid, grid_search.best_estimator_.predict(X_valid)),
        "R²": r2_score(y_valid, grid_search.best_estimator_.predict(X_valid))
    }
}


# Convertimos a DataFrame
results_df = pd.DataFrame(results).T

# --- Gráfico comparativo ---
plt.figure(figsize=(10,6))
results_df[["RMSE","MAE"]].plot(kind="bar", figsize=(10,6))
plt.title("Comparación de RMSE y MAE entre modelos")
plt.ylabel("Error")
plt.xticks(rotation=0)
plt.legend(title="Métrica")
plt.show()

# --- Gráfico R² ---
plt.figure(figsize=(8,5))
sns.barplot(
    x=results_df.index, 
    y=results_df["R²"], 
    hue=results_df.index,   # usamos la variable del eje como hue
    palette="Blues_r", 
    dodge=False, 
    legend=False
)
plt.title("Comparación de R² entre modelos")
plt.ylabel("R²")
plt.ylim(0,1)
plt.show()

results_df

# ===================================
# Guardar resultados en CSV
# ===================================

# ===================================
# Función para guardar métricas estandarizadas
# ===================================
def save_results(model_name, y_true, y_pred, filepath):
    """
    Calcula métricas (RMSE, MAE, R²) y guarda en CSV estandarizado.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    results = pd.DataFrame([{
        "Modelo": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }])


    results.to_csv(filepath, index=False)
    print(f"✅ Resultados de {model_name} guardados en {filepath}")
    return results



# Guardar cada modelo por separado

# ===================================
# Guardar métricas en CSV para comparativa global
# ===================================

# Decision Tree
results_tree = pd.DataFrame([{
    "Modelo": "Decision Tree",
    "RMSE": np.sqrt(mean_squared_error(y_valid, preds_tree)),
    "MAE": mean_absolute_error(y_valid, preds_tree),
    "R²": r2_score(y_valid, preds_tree)
}])
results_tree.to_csv("../data/results_tree.csv", index=False)
print("✅ Resultados Decision Tree guardados en data/results_tree.csv")

# Random Forest (baseline)
results_rf_base = pd.DataFrame([{
    "Modelo": "Random Forest (baseline)",
    "RMSE": np.sqrt(mean_squared_error(y_valid, preds_rf)),
    "MAE": mean_absolute_error(y_valid, preds_rf),
    "R²": r2_score(y_valid, preds_rf)
}])
results_rf_base.to_csv("../data/results_rf_baseline.csv", index=False)
print("✅ Resultados Random Forest (baseline) guardados en data/results_rf_baseline.csv")

# Random Forest (optimizado con GridSearchCV)
preds_rf_opt = grid_search.best_estimator_.predict(X_valid)
results_rf_opt = pd.DataFrame([{
    "Modelo": "Random Forest (optimizado)",
    "RMSE": np.sqrt(mean_squared_error(y_valid, preds_rf_opt)),
    "MAE": mean_absolute_error(y_valid, preds_rf_opt),
    "R²": r2_score(y_valid, preds_rf_opt)
}])
results_rf_opt.to_csv("../data/results_rf_optimized.csv", index=False)
print("✅ Resultados Random Forest (optimizado) guardados en data/results_rf_optimized.csv")

